##Testing function

In [1]:
import json
import sys

def run_generic_tests(func_to_test, file_path):
    """
    Loads test cases from a JSON file and runs them against *any* function.

    Assumes:
    1. The function 'func_to_test' takes numerical arguments (e.g., n, m).
    2. The 'input' in the JSON is a string of space-separated numbers.
    """
    try:
        with open(file_path, 'r') as f:
            test_cases = json.load(f)
    except Exception as e:
        print(f"❌ Error loading '{file_path}': {e}")
        return

    print(f"--- 🏃 Running {len(test_cases)} cases for {func_to_test.__name__} ---")
    passed_count = 0

    for i, case in enumerate(test_cases):
        input_str = case['input']
        expected_output = case['output']

        try:
            args = list(map(int, input_str.split()))
            actual_output = func_to_test(*args)

            if str(actual_output) == expected_output:
                passed_count += 1
                print(f"✅ Case {i+1} PASS: Input '{input_str}' -> Expected: '{expected_output}', Got: '{actual_output}'")
            else:
                print(f"❌ Case {i+1} FAIL: Input '{input_str}' -> Expected: '{expected_output}', Got: '{actual_output}'")

        except Exception as e:
            print(f"💥 ERROR on Case {i+1} (Input: '{input_str}'): {e}")

    print("--- 📊 Summary ---")
    if passed_count == len(test_cases):
        print(f"✅ All {passed_count} cases passed!")
    else:
        print(f"⚠️ {passed_count} / {len(test_cases)} cases passed.")

Iterative 1

In [2]:
def find_day2(n, m):
    # Case 1: n is too small to run out at any point
    if n <= m:
        return 0  # Never runs out

    left, right = 1, 2 * 10**18  # Large enough range

    def used(k):
        # Total used on k days = sum_{i=1}^k i = k*(k+1)/2
        # Extra grain added = m*(k-1)
        return k * (k + 1) // 2 - m * (k - 1)

    while left < right:
        mid = (left + right) // 2
        if used(mid) < n:
            left = mid + 1
        else:
            right = mid

    return left

In [3]:
run_generic_tests(find_day2, './test_cases.json')

--- 🏃 Running 223 cases for find_day2 ---
❌ Case 1 FAIL: Input '5 2' -> Expected: '4', Got: '5'
✅ Case 2 PASS: Input '8 1' -> Expected: '5', Got: '5'
❌ Case 3 FAIL: Input '32 5' -> Expected: '12', Got: '14'
❌ Case 4 FAIL: Input '1024 1024' -> Expected: '1024', Got: '0'
❌ Case 5 FAIL: Input '58044 52909' -> Expected: '53010', Got: '105818'
❌ Case 6 FAIL: Input '996478063 658866858' -> Expected: '658892843', Got: '1317733716'
❌ Case 7 FAIL: Input '570441179141911871 511467058318039545' -> Expected: '511467058661475480', Got: '1022934116636079090'
❌ Case 8 FAIL: Input '1 1' -> Expected: '1', Got: '0'
❌ Case 9 FAIL: Input '1000000000000000000 1000000000000000000' -> Expected: '1000000000000000000', Got: '0'
❌ Case 10 FAIL: Input '1000000000000000000 999999999999997145' -> Expected: '999999999999997221', Got: '1999999999999994290'
❌ Case 11 FAIL: Input '1 1000000000000000000' -> Expected: '1', Got: '0'
✅ Case 12 PASS: Input '1000000000000000000 1' -> Expected: '1414213563', Got: '1414213563

Iterative 2

In [4]:
def find_day3(n, m):
    # Edge case: If refill amount is >= daily consumption, the barn is always full
    # This means: on day 1, m >= 1 → barn replenished
    if m >= n:
        return 0  # Never runs out (always full)

    grain = n  # Initial full barn
    day = 1

    while True:
        # 1. Sparrows eat 'day' grains
        grain -= day
        day += 1

        # 2. Barn refills m grains **only if not already full**
        if grain <= 0:
            return day - 1  # We ran out on previous day
        if grain < n:
            grain = min(grain + m, n)

In [6]:
run_generic_tests(find_day3, './test_cases.json')

--- 🏃 Running 223 cases for find_day3 ---
✅ Case 1 PASS: Input '5 2' -> Expected: '4', Got: '4'
✅ Case 2 PASS: Input '8 1' -> Expected: '5', Got: '5'
✅ Case 3 PASS: Input '32 5' -> Expected: '12', Got: '12'
❌ Case 4 FAIL: Input '1024 1024' -> Expected: '1024', Got: '0'
✅ Case 5 PASS: Input '58044 52909' -> Expected: '53010', Got: '53010'
✅ Case 6 PASS: Input '996478063 658866858' -> Expected: '658892843', Got: '658892843'


KeyboardInterrupt: 

Had to stop because of infinite loop.

Iterative 3

In [7]:
def find_day4(n, m):
    grain = n  # Start with full barn

    for day in range(1, 2 * 10**9):  # Large enough upper bound
        # Step 1: Refill m grains if possible
        grain += min(m, n - grain)

        # Step 2: Subtract k = day grains
        grain -= day

        # Step 3: Check if we're out
        if grain < 0:
            return day

    return -1  # Just in case loop exits without finding

In [8]:
run_generic_tests(find_day3, './test_cases.json')

--- 🏃 Running 223 cases for find_day3 ---
✅ Case 1 PASS: Input '5 2' -> Expected: '4', Got: '4'
✅ Case 2 PASS: Input '8 1' -> Expected: '5', Got: '5'
✅ Case 3 PASS: Input '32 5' -> Expected: '12', Got: '12'
❌ Case 4 FAIL: Input '1024 1024' -> Expected: '1024', Got: '0'
✅ Case 5 PASS: Input '58044 52909' -> Expected: '53010', Got: '53010'
✅ Case 6 PASS: Input '996478063 658866858' -> Expected: '658892843', Got: '658892843'


KeyboardInterrupt: 

Iteration 4

In [9]:
def find_day4(n, m):
    if n <= m:
        return n  # Case 1: Sparrows will deplete the barn in n days

    # Case 2: n > m
    remaining = n - m  # Amount left after Phase 1 (after day m)

    # Binary search to find the smallest k such that k*(k+1)//2 >= remaining
    low, high = 1, int((2 * remaining) ** 0.5) + 1  # Reasonable upper bound

    while low < high:
        mid = (low + high) // 2
        total = mid * (mid + 1) // 2
        if total >= remaining:
            high = mid
        else:
            low = mid + 1

    # Final answer is m (Phase 1 days) + k (Phase 2 days)
    return m + low

In [10]:
run_generic_tests(find_day4, './test_cases.json')

--- 🏃 Running 223 cases for find_day4 ---
✅ Case 1 PASS: Input '5 2' -> Expected: '4', Got: '4'
✅ Case 2 PASS: Input '8 1' -> Expected: '5', Got: '5'
✅ Case 3 PASS: Input '32 5' -> Expected: '12', Got: '12'
✅ Case 4 PASS: Input '1024 1024' -> Expected: '1024', Got: '1024'
✅ Case 5 PASS: Input '58044 52909' -> Expected: '53010', Got: '53010'
✅ Case 6 PASS: Input '996478063 658866858' -> Expected: '658892843', Got: '658892843'
✅ Case 7 PASS: Input '570441179141911871 511467058318039545' -> Expected: '511467058661475480', Got: '511467058661475480'
✅ Case 8 PASS: Input '1 1' -> Expected: '1', Got: '1'
✅ Case 9 PASS: Input '1000000000000000000 1000000000000000000' -> Expected: '1000000000000000000', Got: '1000000000000000000'
✅ Case 10 PASS: Input '1000000000000000000 999999999999997145' -> Expected: '999999999999997221', Got: '999999999999997221'
✅ Case 11 PASS: Input '1 1000000000000000000' -> Expected: '1', Got: '1'
✅ Case 12 PASS: Input '1000000000000000000 1' -> Expected: '1414213563',

Finally on giving exact description of what was wrong, Qwen was able to give the right solution which passed all the test cases.

In [1]:
!pip install -q nbmerge

  Preparing metadata (setup.py) ... done


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!nbmerge '/content/drive/My Drive/nbs/LlamaPart2TestingP1.ipynb' '/content/drive/My Drive/nbs/LlamaPart2TestingP2.ipynb' -o '/content/drive/My Drive/nbs/LlamaPart2Testing.ipynb'